# Deep Supervised Graph Partitioning Model (DSGPM)

This Collab illustrates how to use pre-trained DSGPM model to predict CG mappings. DSGPM is a graph neural network graph partitioning model that can predict CG mappings of small to arbitrarily large molecules. 

## Citation


```
@Article{D0SC02458A,
author ="Li, Zhiheng and Wellawatte, Geemi P. and Chakraborty, Maghesree and Gandhi, Heta A. and Xu, Chenliang and White, Andrew D.",
title  ="Graph neural network based coarse-grained mapping prediction",
journal  ="Chem. Sci.",
year  ="2020",
pages  ="-",
publisher  ="The Royal Society of Chemistry",
doi  ="10.1039/D0SC02458A",
url  ="http://dx.doi.org/10.1039/D0SC02458A",
}

```




## Setup the conda environment and install dependencies

In [1]:
#@title
from IPython.utils import io
import os
import subprocess
import tqdm.notebook

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:

      %shell rm -rf /opt/conda
      %shell wget -q -P /tmp \
        https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
          && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
          && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
      pbar.update(10)

      PATH=%env PATH
      %env PATH=/opt/conda/bin:{PATH}
      %shell conda update -qy conda && conda install -qy -c conda-forge python=3.7
      pbar.update(15)
      #%shell conda install -qy -c pytorch cudatoolkit=10.2 
      pbar.update(15)
      %shell conda install -qy -c conda-forge rdkit
      pbar.update(10)
      %shell git clone https://github.com/rochesterxugroup/DSGPM.git
      pbar.update(10)
      %shell python -m pip install scikit-learn~=0.21.3 numpy~=1.19.1 scipy~=1.3.1  networkx~=2.4 tqdm~=4.47.0
      pbar.update(10)
      %shell conda install -qy cudatoolkit=10.1
      %shell python -m pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
      %shell pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html 
      pbar.update(30)
      
except subprocess.CalledProcessError:
  print(captured)
  raise

## Note:

In case you come across torch-geometry installation errors, it could be due to version mismatch errors. Relevant packages must be installed to resolve such version mismactches. We can do this by printing out the torch and cuda versions prior to instalation. 

For more details please visit PyTorch geometric [documentation!](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html)

# Using DSGPM to generate CG mappings

In this example let's use SMILES strings as inputs to DSGPM. Save one or many SMILES string in text file in your local computer and upload here. If you wish to generate mappings for arbitrarily large molecules, you can use PDB format. Please refer to our [documentation](https://github.com/rochesterxugroup/DSGPM#readme) on how to work with PDB inputs. 




In [2]:
#@title Upload the text file here
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  txt_input = fn

##title Set input and output paths
smiles_path = os.path.join(str(txt_input))
out_path = os.path.join('/content','dsgpm_out')

Saving butanol.txt to butanol.txt


In [3]:
#@title Step 1: Convert your SMILES string into a molecular graph
%shell python /content/DSGPM/generate_input_files/convert_to_json.py --smiles $smiles_path

CCCCO
conversion complete


In [4]:
#@title Step 2: Predict the CG mapping
%shell python /content/DSGPM/inference.py --pretrained_ckpt /content/DSGPM/model/DSGPM_trained.pth  --data_root /content/mol_graph/ --json_output_dir $out_path --num_cg_beads 3 --no_automorphism 

100% 1/1 [00:00<00:00, 12.77it/s]


## To take a look at the output:


```
cd /content/dsgpm_out/dsgpm/
%shell cat <file name>
```

Our [documentation](https://github.com/rochesterxugroup/DSGPM#readme) provides a code snippet to generate SVG images of the outputs. Note: "smiles" field is required in json files to create images.
